# Optimización de un modelo de Random Forest

Este notebook recoge los resultados de la búsqueda del mejor modelo de clasificación mediante Random Forest. El método entrena varios árboles de decisión (de clasificación en este caso) en submuestras de los datos y combina sus resultados para mejorar su precisión.

Para buscar el mejor modelo posible, se tratará de buscar los mejores hiperparámetros para:

* El número de árboles del bosque.
* La profundidad máxima que alcanzan estos.
* Función para evaluar una nueva división de una rama.

### Preparación de los datos

In [2]:
import pandas as pd
import numpy as np
# Data partition
from sklearn.model_selection import train_test_split
# Parameter tunning libraries
import optuna
from sklearn.model_selection import GridSearchCV
# Accuracy function
from sklearn.metrics import accuracy_score
# Model
from sklearn.ensemble import RandomForestClassifier

In [4]:
# Datos de entrenamiento
trainFNC = pd.read_csv("../data/train_FNC.csv")
trainSBM = pd.read_csv("../data/train_SBM.csv")
train_labels = pd.read_csv("../data/train_labels.csv")

# DataFrame con ambas fuentes de datos
train = pd.merge(left=trainFNC, right=trainSBM, left_on='Id', right_on='Id')
data = pd.merge(left=train_labels, right=train, left_on='Id', right_on='Id')
data.drop("Id", inplace=True, axis=1)

# Shuffle de los datos de train
data = data.sample(frac=1, random_state=0)
data.head(5)

,Class,FNC1,FNC2,FNC3,FNC4,FNC5,FNC6,FNC7,FNC8,FNC9,...,SBM_map55,SBM_map61,SBM_map64,SBM_map67,SBM_map69,SBM_map71,SBM_map72,SBM_map73,SBM_map74,SBM_map75
2,0,0.245850,0.216620,-0.124680,-0.353800,0.161500,-0.002032,-0.133020,-0.035222,0.259040,...,-0.257114,0.597229,1.220756,-0.059213,-0.435494,-0.092971,1.090910,-0.448562,-0.508497,0.350434
13,1,0.410730,-0.031925,0.210700,0.242260,0.320100,-0.419290,-0.187140,0.168450,0.599790,...,-0.050862,0.870602,0.609465,1.181878,-2.279469,-0.013484,-0.012693,-1.244346,-1.080442,-0.788502
53,1,0.070919,0.034179,-0.011755,0.019158,0.024645,-0.032022,0.004620,0.318170,0.212550,...,-1.539922,-1.495822,1.643866,1.687780,1.521086,-1.988432,-0.267471,0.510576,1.104566,-1.067206
41,0,0.087377,-0.052462,-0.007835,-0.112830,0.389380,0.216080,0.063572,-0.251230,-0.080568,...,-0.077353,-0.459463,-0.204328,-0.619508,-1.410523,-0.304622,-1.521928,0.593691,0.073638,-0.260920
74,0,0.202750,0.191420,-0.056662,-0.157780,0.244040,0.039780,-0.001503,0.001056,-0.048222,...,0.044457,0.593326,1.063052,0.434726,1.604964,-0.359736,0.210107,0.355922,0.730287,-0.323557


Vamos a usar la siguiente partición de los datos:

* 60% train $\sim$ 50 datos
* 20% validation $\sim$ 18 datos (se define al aplicar cross-validación en el ajuste)
* 20% test $\sim$ 18 datos

In [5]:
X = data.iloc[:, 1:]
y = data.iloc[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("Tamaño del dataset de train:", X_train.shape)
print("Tamaño del dataset de test:", X_test.shape)

Tamaño del dataset de train: (68, 410)
Tamaño del dataset de test: (18, 410)


In [7]:
# Datos de test
testFNC = pd.read_csv("../data/test_FNC.csv")
testSBM = pd.read_csv("../data/test_SBM.csv")

# DataFrame con ambas fuentes de datos
test = pd.merge(left=testFNC, right=testSBM, left_on='Id', right_on='Id')
test.drop("Id", inplace=True, axis=1)
test.head(5)

,FNC1,FNC2,FNC3,FNC4,FNC5,FNC6,FNC7,FNC8,FNC9,FNC10,...,SBM_map55,SBM_map61,SBM_map64,SBM_map67,SBM_map69,SBM_map71,SBM_map72,SBM_map73,SBM_map74,SBM_map75
0,0.476127,0.064466,0.053238,-0.608133,0.073988,-0.637038,0.113556,-0.192434,-0.004025,-0.060474,...,-0.451994,1.123770,2.083006,1.145440,-0.067608,1.202529,0.851587,0.451583,-0.159739,0.192076
1,0.013833,0.267183,0.232178,-0.167151,-0.261327,0.191869,0.406493,0.088761,0.177048,0.036718,...,0.696987,1.397832,1.046136,-0.191733,-2.192023,-0.369276,0.822225,-0.109342,-0.580476,0.174160
2,-0.435452,0.046780,0.243742,0.397030,-0.147821,0.173620,-0.461963,-0.610736,0.419753,0.400985,...,0.160145,1.906989,-2.661633,-0.193911,0.440873,0.641739,0.918397,-0.758046,0.154701,-0.476647
3,-0.204510,-0.036735,-0.760705,-0.740495,0.064668,0.349926,-0.273826,-0.174384,-0.120248,0.175618,...,0.974828,-1.997087,-2.083782,1.154107,-0.643947,2.332424,0.659124,-0.809445,0.558960,2.790871
4,0.599435,-0.166441,0.122431,0.011539,0.346906,-0.017430,-0.274734,0.211510,0.151012,-0.033434,...,-0.789153,1.578984,1.402592,-1.230440,0.296686,2.806314,0.427184,-0.240682,-0.196948,-1.544345


### Modelo

In [8]:
def train_model(model, param_grid):
    '''Función para realizar el entrenamiento y la búsqueda de hiperparámetros'''
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=4)
    # cv = 4 porque así: el conjunto de validation tiene un 0.25 del tamaño de train y: 0.25 * 0.8 = 0.2 ~ 20% datos
    #                    el conjunto de train tiene un 0.75 del tamaño de train y: 0.75 * 0.8 = 0.6 ~ 60% datos
    grid_search.fit(X_train, y_train)
    
    print("Parámetros óptimos:", grid_search.best_params_)
    print("Modelo óptimo:", grid_search.best_estimator_)
    
    return grid_search.best_estimator_

Primera prueba de resultados, probando con valores concretos para el número de árboles y la profundidad de los mismos (parámetros ``n_estimators`` y ``max_depth`` respectivamente) y utilizando el método ``GridSearchCV`` de ``sklearn`` para realizar la búsqueda:

In [9]:
# Definir y entrenar el modelo
model_RF = RandomForestClassifier(random_state=0)
param_grid_RF = {
    "n_estimators": [100, 250, 500, 750, 1000],
    "criterion": ["gini", "entropy"],
    "max_depth": [5, 10, 15, 20, None]
}
model_RF_opt = train_model(model_RF, param_grid_RF)

# Predicción en partición de test
y_pred_RF = model_RF_opt.predict(X_test)

# Precisión en partición de test
accuracy = accuracy_score(y_test, y_pred_RF)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

# Predicción en test para kaggle
y_pred_kaggle_RF = model_RF_opt.predict(test)

Parámetros óptimos: {'criterion': 'entropy', 'max_depth': 5, 'n_estimators': 750}
Modelo óptimo: RandomForestClassifier(criterion='entropy', max_depth=5, n_estimators=750,
                       random_state=0)
Accuracy: 83.33%


Segunda prueba, con un rango más amplio de valores:

In [10]:
# Definir y entrenar el modelo
model_RF = RandomForestClassifier(random_state=0)
param_grid_RF = {
    "n_estimators": range(50, 1050, 50),
    "criterion": ["gini", "entropy"],
    "max_depth": range(1, 21)
}
model_RF_opt = train_model(model_RF, param_grid_RF)

# Predicción en partición de test
y_pred_RF = model_RF_opt.predict(X_test)

# Precisión en partición de test
accuracy = accuracy_score(y_test, y_pred_RF)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

# Predicción en test para kaggle
y_pred_kaggle_RF = model_RF_opt.predict(test)

Parámetros óptimos: {'criterion': 'entropy', 'max_depth': 4, 'n_estimators': 600}
Modelo óptimo: RandomForestClassifier(criterion='entropy', max_depth=4, n_estimators=600,
                       random_state=0)
Accuracy: 72.22%


$\color{red}{\text{¿POR QUÉ EMPEORA?}}$

La librería ``optuna`` es un framework específico para la optimización de hiperparámetros, repetiremos el proceso de búsqueda anterior utilizando esta librería para ver el rendimiento, se espera que los resultados sean iguales si no mejores.

In [11]:
def objectiveRF(trial):
    
    n_estimators =  trial.suggest_int("n_estimators", 50, 1000, 50) # optuna incluye en el rango el máximo y el mínimo
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    max_depth = trial.suggest_int("max_depth", 1, 20)
    
    modelRF_optuna = RandomForestClassifier(criterion = criterion, max_depth = max_depth, n_estimators = n_estimators, 
                                            random_state=0)
    
    modelRF_optuna.fit(X_train, y_train)

    y_pred_RF_optuna = modelRF_optuna.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_RF_optuna)
    return accuracy

In [12]:
sampler = optuna.samplers.TPESampler(seed=0)  # Asegurar los reproducibilidad de los resultados
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objectiveRF, n_trials=800)
# n_trials = 20 x 2 x 20 = 800

[I 2022-05-23 18:38:24,558] A new study created in memory with name: no-name-01857f30-d71f-4367-a0dc-613e3f255b85
[I 2022-05-23 18:38:25,698] Trial 0 finished with value: 0.6666666666666666 and parameters: {'n_estimators': 550, 'criterion': 'gini', 'max_depth': 11}. Best is trial 0 with value: 0.6666666666666666.
[I 2022-05-23 18:38:26,282] Trial 1 finished with value: 0.6666666666666666 and parameters: {'n_estimators': 450, 'criterion': 'gini', 'max_depth': 18}. Best is trial 0 with value: 0.6666666666666666.
[I 2022-05-23 18:38:27,747] Trial 2 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 1000, 'criterion': 'entropy', 'max_depth': 11}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:38:28,432] Trial 3 finished with value: 0.6666666666666666 and parameters: {'n_estimators': 600, 'criterion': 'gini', 'max_depth': 2}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:38:28,523] Trial 4 finished with value: 0.7777777777777778 and

[I 2022-05-23 18:39:08,911] Trial 38 finished with value: 0.7222222222222222 and parameters: {'n_estimators': 800, 'criterion': 'gini', 'max_depth': 16}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:39:10,301] Trial 39 finished with value: 0.7222222222222222 and parameters: {'n_estimators': 950, 'criterion': 'entropy', 'max_depth': 2}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:39:11,300] Trial 40 finished with value: 0.7222222222222222 and parameters: {'n_estimators': 650, 'criterion': 'gini', 'max_depth': 5}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:39:12,802] Trial 41 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 950, 'criterion': 'entropy', 'max_depth': 12}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:39:14,253] Trial 42 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 950, 'criterion': 'entropy', 'max_depth': 15}. Best is trial 2 with value: 0.8

[I 2022-05-23 18:39:55,325] Trial 76 finished with value: 0.8333333333333334 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 4}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:39:56,830] Trial 77 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 1000, 'criterion': 'entropy', 'max_depth': 7}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:39:58,240] Trial 78 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 14}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:39:59,589] Trial 79 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 15}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:40:00,872] Trial 80 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 850, 'criterion': 'entropy', 'max_depth': 15}. Best is trial 2 with val

[I 2022-05-23 18:40:44,688] Trial 114 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 850, 'criterion': 'entropy', 'max_depth': 16}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:40:46,196] Trial 115 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 950, 'criterion': 'entropy', 'max_depth': 14}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:40:47,598] Trial 116 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 950, 'criterion': 'entropy', 'max_depth': 14}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:40:48,963] Trial 117 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 950, 'criterion': 'entropy', 'max_depth': 18}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:40:50,424] Trial 118 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 1000, 'criterion': 'entropy', 'max_depth': 13}. Best is trial 2 w

[I 2022-05-23 18:41:37,938] Trial 152 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 14}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:41:39,247] Trial 153 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 14}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:41:40,555] Trial 154 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 16}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:41:42,035] Trial 155 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 1000, 'criterion': 'entropy', 'max_depth': 7}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:41:43,613] Trial 156 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 1000, 'criterion': 'entropy', 'max_depth': 10}. Best is trial 2 w

...

[I 2022-05-23 18:56:43,871] Trial 646 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 850, 'criterion': 'entropy', 'max_depth': 10}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:56:45,136] Trial 647 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 850, 'criterion': 'entropy', 'max_depth': 20}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:56:46,034] Trial 648 finished with value: 0.7777777777777778 and parameters: {'n_estimators': 600, 'criterion': 'entropy', 'max_depth': 20}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:56:47,302] Trial 649 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 850, 'criterion': 'entropy', 'max_depth': 9}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:56:48,322] Trial 650 finished with value: 0.7777777777777778 and parameters: {'n_estimators': 650, 'criterion': 'entropy', 'max_depth': 10}. Best is trial 2 wit

[I 2022-05-23 18:57:40,502] Trial 684 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 17}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:57:43,515] Trial 685 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 17}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:57:46,363] Trial 686 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 17}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:57:49,084] Trial 687 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 17}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:57:52,240] Trial 688 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 18}. Best is trial 2 wi

[I 2022-05-23 18:58:56,074] Trial 722 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 17}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:58:57,578] Trial 723 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 17}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:58:59,423] Trial 724 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 950, 'criterion': 'entropy', 'max_depth': 18}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:59:01,181] Trial 725 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 17}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:59:02,581] Trial 726 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 17}. Best is trial 2 wi

[I 2022-05-23 18:59:53,897] Trial 760 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 950, 'criterion': 'entropy', 'max_depth': 18}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:59:55,387] Trial 761 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 1000, 'criterion': 'entropy', 'max_depth': 19}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:59:57,175] Trial 762 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 1000, 'criterion': 'entropy', 'max_depth': 11}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 18:59:59,921] Trial 763 finished with value: 0.5555555555555556 and parameters: {'n_estimators': 1000, 'criterion': 'entropy', 'max_depth': 1}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 19:00:02,431] Trial 764 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 16}. Best is trial 2 

[I 2022-05-23 19:01:17,828] Trial 798 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 1000, 'criterion': 'entropy', 'max_depth': 19}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-05-23 19:01:19,193] Trial 799 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 14}. Best is trial 2 with value: 0.8888888888888888.


In [13]:
study.best_trial

FrozenTrial(number=2, values=[0.8888888888888888], datetime_start=datetime.datetime(2022, 5, 23, 18, 38, 26, 282051), datetime_complete=datetime.datetime(2022, 5, 23, 18, 38, 27, 731799), params={'n_estimators': 1000, 'criterion': 'entropy', 'max_depth': 11}, distributions={'n_estimators': IntUniformDistribution(high=1000, low=50, step=50), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_depth': IntUniformDistribution(high=20, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=2, state=TrialState.COMPLETE, value=None)

In [14]:
# Definir y entrenar el modelo
modelRF_optuna = RandomForestClassifier(criterion = "entropy", max_depth = 11, n_estimators = 1000, random_state=0)  
modelRF_optuna.fit(X_train, y_train)

# Predicción en partición de test
y_pred_RF_optuna = modelRF_optuna.predict(X_test)

# Precisión en partición de test
accuracy = accuracy_score(y_test, y_pred_RF_optuna)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

Accuracy: 88.89%


La librería ``optuna`` obtiene mejores resultados.

El código anterior, aunque realiza una búsqueda sobre el mismo rango de parámetros usados para el segundo intento con ``sklearn``, no está aplicando cross-validation an el entrenamiento. La siguiente celda sí lo implementa mediante el método ``OptunaSearchCV`` de ``optuna``:

In [16]:
# Definir y entrenar el modelo
model_RF = RandomForestClassifier(random_state=0)
param_grid_RF = {
    "n_estimators": optuna.distributions.IntUniformDistribution(50, 1000, 50),
    "criterion": optuna.distributions.CategoricalDistribution(["gini", "entropy"]),
    "max_depth": optuna.distributions.IntUniformDistribution(1, 20)
}

optuna_search = optuna.integration.OptunaSearchCV(model_RF, param_grid_RF, cv=4, n_trials=800, random_state=0)
# n_trials = 20 x 2 x 20 = 800
optuna_search.fit(X_train, y_train)

C:\Users\saral\AppData\Local\Temp\ipykernel_15376\3155617736.py:9: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(model_RF, param_grid_RF, cv=4, n_trials=800, random_state=0)
[I 2022-05-23 19:05:42,765] A new study created in memory with name: no-name-c59f6910-0389-4284-997d-cb1731c80c85
[I 2022-05-23 19:05:45,917] Trial 0 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 500, 'criterion': 'entropy', 'max_depth': 4}. Best is trial 0 with value: 0.6911764705882353.
[I 2022-05-23 19:05:48,713] Trial 1 finished with value: 0.6029411764705883 and parameters: {'n_estimators': 600, 'criterion': 'gini', 'max_depth': 15}. Best is trial 0 with value: 0.6911764705882353.
[I 2022-05-23 19:05:53,844] Trial 2 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 850, 'criterion': 'entropy', 'max_depth': 11}. Best is trial 0 with value: 

[I 2022-05-23 19:08:04,067] Trial 36 finished with value: 0.5882352941176471 and parameters: {'n_estimators': 600, 'criterion': 'gini', 'max_depth': 2}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 19:08:09,282] Trial 37 finished with value: 0.6617647058823529 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 5}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 19:08:12,466] Trial 38 finished with value: 0.6617647058823529 and parameters: {'n_estimators': 550, 'criterion': 'entropy', 'max_depth': 8}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 19:08:12,779] Trial 39 finished with value: 0.6323529411764707 and parameters: {'n_estimators': 50, 'criterion': 'gini', 'max_depth': 11}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 19:08:18,048] Trial 40 finished with value: 0.6617647058823529 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 15}. Best is trial 12 with value: 

[I 2022-05-23 19:10:37,269] Trial 74 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 18}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 19:10:40,977] Trial 75 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 700, 'criterion': 'entropy', 'max_depth': 17}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 19:10:45,191] Trial 76 finished with value: 0.6617647058823529 and parameters: {'n_estimators': 850, 'criterion': 'gini', 'max_depth': 18}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 19:10:49,873] Trial 77 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 16}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 19:10:53,813] Trial 78 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 650, 'criterion': 'entropy', 'max_depth': 6}. Best is trial 12 with v

[I 2022-05-23 19:14:02,651] Trial 112 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 750, 'criterion': 'entropy', 'max_depth': 19}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 19:14:07,114] Trial 113 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 750, 'criterion': 'entropy', 'max_depth': 13}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 19:14:10,754] Trial 114 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 700, 'criterion': 'entropy', 'max_depth': 13}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 19:14:14,915] Trial 115 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 700, 'criterion': 'entropy', 'max_depth': 14}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 19:14:19,611] Trial 116 finished with value: 0.676470588235294 and parameters: {'n_estimators': 850, 'criterion': 'entropy', 'max_depth': 4}. Best is trial 12

[I 2022-05-23 19:16:42,932] Trial 150 finished with value: 0.6470588235294118 and parameters: {'n_estimators': 800, 'criterion': 'gini', 'max_depth': 18}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 19:16:47,009] Trial 151 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 750, 'criterion': 'entropy', 'max_depth': 13}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 19:16:50,846] Trial 152 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 700, 'criterion': 'entropy', 'max_depth': 14}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 19:16:55,194] Trial 153 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 750, 'criterion': 'entropy', 'max_depth': 11}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 19:16:59,950] Trial 154 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 850, 'criterion': 'entropy', 'max_depth': 11}. Best is trial 12 

...

[I 2022-05-23 22:40:01,012] Trial 683 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 850, 'criterion': 'entropy', 'max_depth': 14}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 22:40:08,582] Trial 684 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 750, 'criterion': 'entropy', 'max_depth': 16}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 22:40:16,757] Trial 685 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 20}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 22:40:26,141] Trial 686 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 14}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 22:40:36,426] Trial 687 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 850, 'criterion': 'entropy', 'max_depth': 14}. Best is trial 

[I 2022-05-23 22:45:40,904] Trial 721 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 700, 'criterion': 'entropy', 'max_depth': 18}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 22:45:49,088] Trial 722 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 700, 'criterion': 'entropy', 'max_depth': 18}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 22:45:59,475] Trial 723 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 850, 'criterion': 'entropy', 'max_depth': 15}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 22:46:06,117] Trial 724 finished with value: 0.5735294117647058 and parameters: {'n_estimators': 600, 'criterion': 'entropy', 'max_depth': 1}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 22:46:16,493] Trial 725 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 850, 'criterion': 'entropy', 'max_depth': 15}. Best is trial 1

[I 2022-05-23 22:51:18,764] Trial 759 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 20}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 22:51:27,395] Trial 760 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 20}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 22:51:35,684] Trial 761 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 750, 'criterion': 'entropy', 'max_depth': 10}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 22:51:43,157] Trial 762 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 700, 'criterion': 'entropy', 'max_depth': 10}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 22:51:51,626] Trial 763 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 750, 'criterion': 'entropy', 'max_depth': 19}. Best is trial 

[I 2022-05-23 22:57:17,088] Trial 797 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 6}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 22:57:26,962] Trial 798 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 7}. Best is trial 12 with value: 0.7058823529411764.
[I 2022-05-23 22:57:36,480] Trial 799 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 5}. Best is trial 12 with value: 0.7058823529411764.


OptunaSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0),
               n_trials=800,
               param_distributions={'criterion': CategoricalDistribution(choices=('gini', 'entropy')),
                                    'max_depth': IntUniformDistribution(high=20, low=1, step=1),
                                    'n_estimators': IntUniformDistribution(high=1000, low=50, step=50)},
               random_state=0)

In [19]:
# Predicción en partición de test
y_pred_RF = optuna_search.predict(X_test)

# Precisión en partición de test
accuracy = accuracy_score(y_test, y_pred_RF)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

Accuracy: 83.33%


En este caso, aplicar cross-validation no mejora los resultados.

$\color{red}{\text{CÓMO PUEDE SALIR MEJOR PROBANDO MENOS PARÁMETROS??}}$

In [20]:
# Definir y entrenar el modelo
model_RF = RandomForestClassifier(random_state=0)
param_grid_RF = {
    "n_estimators": optuna.distributions.IntUniformDistribution(100, 1000, 100),
    "criterion": optuna.distributions.CategoricalDistribution(["gini", "entropy"]),
    "max_depth": optuna.distributions.IntUniformDistribution(1, 20)
}

optuna_search = optuna.integration.OptunaSearchCV(model_RF, param_grid_RF, cv=4, n_trials=400, random_state=0)
# n_trials = 10 x 2 x 20 = 400
optuna_search.fit(X_train, y_train)

C:\Users\saral\AppData\Local\Temp\ipykernel_15376\3890040142.py:9: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(model_RF, param_grid_RF, cv=4, n_trials=400, random_state=0)
[I 2022-05-23 23:45:04,719] A new study created in memory with name: no-name-61b1a9ed-119c-4357-8782-1e81401a437b
[I 2022-05-23 23:45:07,973] Trial 0 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 500, 'criterion': 'entropy', 'max_depth': 4}. Best is trial 0 with value: 0.6911764705882353.
[I 2022-05-23 23:45:12,264] Trial 1 finished with value: 0.6029411764705883 and parameters: {'n_estimators': 600, 'criterion': 'gini', 'max_depth': 15}. Best is trial 0 with value: 0.6911764705882353.
[I 2022-05-23 23:45:17,797] Trial 2 finished with value: 0.6617647058823529 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 11}. Best is trial 0 with value: 

[I 2022-05-23 23:47:27,489] Trial 36 finished with value: 0.676470588235294 and parameters: {'n_estimators': 900, 'criterion': 'gini', 'max_depth': 16}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-23 23:47:33,912] Trial 37 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 18}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-23 23:47:41,827] Trial 38 finished with value: 0.6764705882352942 and parameters: {'n_estimators': 1000, 'criterion': 'entropy', 'max_depth': 19}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-23 23:47:46,713] Trial 39 finished with value: 0.6470588235294118 and parameters: {'n_estimators': 800, 'criterion': 'gini', 'max_depth': 15}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-23 23:47:52,994] Trial 40 finished with value: 0.6617647058823529 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 19}. Best is trial 28 with val

[I 2022-05-23 23:51:00,925] Trial 74 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 18}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-23 23:51:05,159] Trial 75 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 700, 'criterion': 'entropy', 'max_depth': 14}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-23 23:51:10,908] Trial 76 finished with value: 0.676470588235294 and parameters: {'n_estimators': 900, 'criterion': 'gini', 'max_depth': 18}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-23 23:51:16,602] Trial 77 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 13}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-23 23:51:21,532] Trial 78 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 700, 'criterion': 'entropy', 'max_depth': 13}. Best is trial 28 with v

[I 2022-05-23 23:54:18,060] Trial 112 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 19}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-23 23:54:23,370] Trial 113 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 10}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-23 23:54:28,704] Trial 114 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 18}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-23 23:54:34,265] Trial 115 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 700, 'criterion': 'entropy', 'max_depth': 18}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-23 23:54:40,400] Trial 116 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 700, 'criterion': 'entropy', 'max_depth': 15}. Best is trial 

[I 2022-05-23 23:57:40,071] Trial 150 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 17}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-23 23:57:45,289] Trial 151 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 11}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-23 23:57:50,502] Trial 152 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 11}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-23 23:57:55,329] Trial 153 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 11}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-23 23:58:00,335] Trial 154 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 15}. Best is trial 

...

[I 2022-05-24 00:10:13,114] Trial 264 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 8}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-24 00:10:18,766] Trial 265 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 5}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-24 00:10:24,444] Trial 266 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 8}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-24 00:10:30,824] Trial 267 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 6}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-24 00:10:36,488] Trial 268 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 6}. Best is trial 28 wi

[I 2022-05-24 00:13:25,179] Trial 302 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 8}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-24 00:13:31,058] Trial 303 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 11}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-24 00:13:39,236] Trial 304 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 11}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-24 00:13:45,171] Trial 305 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 11}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-24 00:13:50,798] Trial 306 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 11}. Best is trial 2

[I 2022-05-24 00:18:47,625] Trial 340 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 10}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-24 00:18:56,697] Trial 341 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 20}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-24 00:19:06,218] Trial 342 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 11}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-24 00:19:14,021] Trial 343 finished with value: 0.6911764705882353 and parameters: {'n_estimators': 700, 'criterion': 'entropy', 'max_depth': 10}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-24 00:19:24,771] Trial 344 finished with value: 0.6617647058823529 and parameters: {'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 20}. Best is trial 

[I 2022-05-24 00:24:08,190] Trial 378 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 17}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-24 00:24:18,707] Trial 379 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 6}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-24 00:24:27,410] Trial 380 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 6}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-24 00:24:36,243] Trial 381 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 18}. Best is trial 28 with value: 0.7058823529411764.
[I 2022-05-24 00:24:45,048] Trial 382 finished with value: 0.7058823529411764 and parameters: {'n_estimators': 800, 'criterion': 'entropy', 'max_depth': 17}. Best is trial 28

OptunaSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0),
               n_trials=400,
               param_distributions={'criterion': CategoricalDistribution(choices=('gini', 'entropy')),
                                    'max_depth': IntUniformDistribution(high=20, low=1, step=1),
                                    'n_estimators': IntUniformDistribution(high=1000, low=100, step=100)},
               random_state=0)

In [21]:
y_pred_RF = optuna_search.predict(X_test)

# Precisión en partición de test
accuracy = accuracy_score(y_test, y_pred_RF)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

Accuracy: 88.89%


# Create submissions

In [ ]:
import pathlib
from datetime import datetime

def create_submission(pred, method, test_id=testFNC["Id"]):
    submissionDF = pd.DataFrame(list(zip(test_id, pred)), columns=["Id", "Probability"])
    print(submissionDF.shape) # Comprobación del tamaño, debe ser: (119748, 2)
    current_time = datetime.now().strftime("%d-%m-%Y_%Hh%Mmin")
    current_path = pathlib.Path().resolve()
    parent_path = current_path.parent
    submissionDF.to_csv(f"{parent_path}\submissions\MLSP_submission_{method}_{current_time}.csv", header=True, index=False)